In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

/home/death-reaper/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist_data = input_data.read_data_sets("mnist_hand_written_data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist_hand_written_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist_hand_written_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_hand_written_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_hand_written_data/t10k-labels-idx1-ubyte.gz


In [4]:
type(mnist_data)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [5]:
mnist_data

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f6b66452f98>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f6b66452e48>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f6b66452e80>)

In [6]:
def descriminator(X_img, is_var_scope_exist=False):
    
    if is_var_scope_exist:
        tf.get_variable_scope.reuse_variables()
    
    ## First convolutional neural network
    desc_weight0 = tf.get_variable("desc_weight0", 
                                  [5, 5, 1, 32], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.01))
    desc_bias0 = tf.get_variable("desc_bias0", 
                                [32], 
                                initializer = tf.constant_initializer(value=0))
    
    desc_conv_weight0 = tf.nn.conv2d(input=X_img, filter=desc_weight0, stides = [1, 1, 1, 1], padding="SAME")
    
    desc_conv_weight0 = desc_conv_weight0 + desc_bias0
    desc_activated0 = tf.nn.relu(desc_conv_weight0)
    
    desc_conv_net0 = tf.nn.avg_pool(desc_activated0, 
                                 ksize=[1, 2, 2, 1], 
                                 strides = [1, 2, 2, 1], 
                                 padding="SAME")
    
    ## Second convolutional neural network
    desc_weight1 = tf.get_variable("desc_weight1", 
                                   [5, 5, 32, 64], 
                                   initializer = tf.truncated_normal_initializer(stddev = 0.01))
    
    desc_bias1 = tf.get_variable("desc_bias1", 
                                 [64], 
                                 initializer=tf.constant_initializer(value=0))
    
    desc_conv_weight1 = tf.nn.conv2d(input=des_conv_net0, 
                                     filter = desc_weight1, 
                                     strides = [1, 1, 1, 1], 
                                     padding = "SAME")
    desc_conv_weight1 = desc_conv_weight1 + desc_bias1
    desc_activated1 = tf.nn.relu(desc_conv_weight1)
    desc_conv_net1 = tf.nn.avg_pool(input=desc_activated1, 
                                     ksize=[1,2,2,1], 
                                     strides=[1,2,2,1], 
                                     padding="SAME")
    
    
    ## Third convolutional neural network
    desc_weight2 = tf.get_variable("desc_weight2", 
                                   [5, 5, 64, 128], 
                                   initializer = tf.truncated_normal_initializer(stddev = 0.01))
    
    desc_bias2 = tf.get_variable("desc_bias2", 
                                 [128], 
                                 initializer=tf.constant_initializer(value=0))
    
    desc_conv_weight2 = tf.nn.conv2d(input=des_conv_net1, 
                                     filter = desc_weight2, 
                                     strides = [1, 1, 1, 1], 
                                     padding = "SAME")
    desc_conv_weight2 = desc_conv_weight2 + desc_bias2
    desc_activated2 = tf.nn.relu(desc_conv_weight2)
    desc_conv_net2 = tf.nn.avg_pool(input=desc_activated2, 
                                     ksize=[1,2,2,1], 
                                     strides=[1,2,2,1], 
                                     padding="SAME")
    
    ## First fully connected layer
    desc_weight3 = tf.get_variable("desc_weight3", 
                                   [3.5*3.5*128, 1024], 
                                   initializer=tf.truncated_normal_initializer(stddev=0.01))
    desc_bias3 = tf.get_variable("desc_bias3", 
                                 [1024], 
                                 initializer=tf.constant_initializer(value = 0))
    desc_conv_net2_reshaped = tf.reshape(desc_conv_net2, [-1, 3.5*3.5*128])
    
    desc_fully_con1 = tf.matmul(desc_conv_net2_reshaped, desc_weight3)
    desc_fully_con1 = desc_fully_con1 + desc_bias3
    desc_fully_con1 = tf.nn.relu(desc_fully_con1)
    
    ## Second fully connected layer
    desc_weight4 = tf.get_variable("desc_weight4", 
                                   [1024, 128], 
                                   initializer=tf.truncated_normal_initializer(stddev=0.01))
    desc_bias4 = tf.get_variable("desc_bias4", 
                                 [128], 
                                 initializer=tf.constant_initializer(value = 0))
    
    desc_fully_con2 = tf.matmul(desc_fully_con1, desc_weight4)
    desc_fully_con2 = desc_fully_con2 + desc_bias4
    
    
    ## Third fully connected layer
    desc_weight5 = tf.get_variable("desc_weight5", 
                                   [128, 1], 
                                   initializer=tf.truncated_normal_initializer(stddev=0.01))
    desc_bias5 = tf.get_variable("desc_bias5", 
                                 [1], 
                                 initializer=tf.constant_initializer(value = 0))
    
    desc_fully_con3 = tf.matmul(desc_fully_con2, desc_weight5)
    desc_fully_con3 = desc_fully_con3 + desc_bias5
    
    return desc_fully_con3


In [ ]:
def generator(batch_size, noise_dim):
    noise = tf.truncated_normal([batch_size, noise_dim], name = "noise")
    
    ## deconvolutional neural network 0
    gen_weight0 = tf.get_variable("gen_weight0", 
                                  [noise_dim, 3136], 
                                  initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    gen_bias0 = tf.get_variable("gen_bias0", 
                                [3136], 
                                initializer = tf.truncated_normal_initializer(stddev = 0.01))
    gen_network0 = tf.matmul(noise, gen_weight0) + gen_bias0
    gen_network0 = tf.reshape(gen_network0, [-1, 56, 56, 1])
    gen_network0 = tf.layers.batch_normalization(gen_network0, epsilon=1e-5, scope = "bn1")
    gen_network0 = tf.nn.relu(gen_network0)
    
    ## deconvolutional neural network 1
    gen_weight1 = tf.get_variable("gen_weight1", 
                                  [noise_dim, 3136], 
                                  initializer=tf.truncated_normal_initializer(stddev = 0.01))
    
    gen_bias0 = tf.get_variable("gen_bias0", 
                                [3136], 
                                initializer = tf.truncated_normal_initializer(stddev = 0.01))